# StatefulSets (Stateful workloads)

A **StatefulSet** manages pods that need **stable identity** (names, network) and **stable storage**. It is used for stateful systems like databases when you need predictable pod naming and per-replica volumes.


## Why it is used
- Stable pod names: `db-0`, `db-1`, `db-2`.
- Stable DNS identity per pod (often via a headless Service).
- Stable storage: each replica gets its own PVC.
- Ordered scaling/updates when required.


## Common pattern
```text
Headless Service + StatefulSet + volumeClaimTemplates
```

The headless Service provides stable DNS entries like `db-0.db.default.svc`.


## YAML template (pseudo)
```yaml
apiVersion: v1
kind: Service
metadata:
  name: db
spec:
  clusterIP: None
  selector:
    app: db
  ports:
    - port: 5432
      name: db
---
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: db
spec:
  serviceName: db
  replicas: 3
  selector:
    matchLabels:
      app: db
  template:
    metadata:
      labels:
        app: db
    spec:
      containers:
        - name: db
          image: example/db:1.0.0
          volumeMounts:
            - name: data
              mountPath: /var/lib/db
  volumeClaimTemplates:
    - metadata:
        name: data
      spec:
        accessModes: ["ReadWriteOnce"]
        storageClassName: standard
        resources:
          requests:
            storage: 20Gi
```


## Pitfalls
- StatefulSets do not replace proper database operations (backups, upgrades, failover strategy).
- Storage and networking choices are critical; test in realistic environments.
- Rolling updates for stateful systems can require application-specific orchestration.

## References
- StatefulSets: https://kubernetes.io/docs/concepts/workloads/controllers/statefulset/
